# Topics of Mobile Application Reviews

Topics of Mobile Application Reviews is an NLP based method to classify mobile application reviews over prominent set of categories based on description.
User Experience, Safety and Security, Functional stability, and Ease of use. Deep learning based simple transformer learners are used to classify the 
review into one of the above categories.

This sample notebook shows you how to deploy [Topics of Mobile Application Reviews]

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.
 
#### Prerequisite
1. **Note:** This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
   1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used:
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe** 
   1. or your AWS account has a subscription to Topics of Mobile Application Reviews. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.Output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page **topic-extraction-apps-review**
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [32]:
model_package_arn='arn:aws:sagemaker:us-east-2:786796469737:model-package/topic-extraction-apps-review'

In [33]:
import base64
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
#from urllib.parse import urlparse
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
#import urllib.request
import numpy as np
import os

In [34]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-2-786796469737'

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [35]:
model_name='topic-extraction-apps-review'

content_type='text/csv'

real_time_inference_instance_type='ml.m5.xlarge'
batch_transform_inference_instance_type='ml.m5.large'

#### A. Create an endpoint

In [36]:
def predict_wrapper(endpoint, session):
    return sage.predictor.RealTimePredictor(endpoint, session,content_type)

#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

#Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-----------!

The class RealTimePredictor has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


#### B. Create input payload

In [37]:
#Please create the folder structure as mentioned below
import pandas as pd
file_name   = './Mobile-App-review/Input/real/sample_input.csv'

#### C. Perform real-time inference

In [38]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name 'topic-extraction-apps-review' \
    --body fileb://$file_name \
    --content-type 'text/csv' \
    --region us-east-2 \
    ./Mobile-App-review/Output/real/sample_output.csv

{
    "ContentType": "text/csv; charset=utf-8",
    "InvokedProductionVariant": "AllTraffic"
}


#### D.Output
Output will be generated with the name **sample_output.csv**; this file will contain the predefined category number and category name for each review.

#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [39]:
predictor=sage.predictor.RealTimePredictor(model_name, sagemaker_session,content_type)
predictor.delete_endpoint(delete_endpoint_config=True)

The class RealTimePredictor has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [40]:
#upload the batch-transform job input files to S3
transform_input_folder = "./Mobile-App-review/Input/batch"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name) 
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-786796469737/topic-extraction-apps-review


In [41]:
#Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()
print("Batch Transform complete")

.......................... * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
169.254.255.130 - - [15/Apr/2021 19:15:50] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [15/Apr/2021 19:15:50] "#033GET /execution-parameters HTTP/1.1#033" 404 -
#015  0%|          | 0/1 [00:00<?, ?it/s]#015100%|##########| 1/1 [00:00<00:00,  3.96it/s]#015100%|##########| 1/1 [00:00<00:00,  3.96it/s]

#015  0%|          | 0/1 [00:00<?, ?it/s]#015100%|##########| 1/1 [00:00<00:00,  3.88it/s]#015100%|##########| 1/1 [00:00<00:00,  3.88it/s]
2021-04-15T19:15:50.742:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
#015  0%|          | 0/1 [00:00<?, ?it/s]#015100%|##########| 1/1 [00:00<00:00,  3.93it/s]#015100%|##########| 1/1 [00:00<00:00,  3.93it/s]
a
b
c
d
169.254.255.130 - - [15/Apr/2021 19:15:57] "#033POST /invocations HTTP

UnexpectedStatusException: Error for Transform job topic-extraction-apps-review-2021-04-15-19-11-35-709: Failed. Reason: AlgorithmError: See job logs for more information

In [42]:
#output is available on following path
transformer.output_path

's3://sagemaker-us-east-2-786796469737/topic-extraction-apps-review-2021-04-15-19-11-35-709'

In [43]:
s3_conn = boto3.client("s3")
bucket_name="sagemaker-us-east-2-786796469737"
with open('./Mobile-App-review/Output/batch/sample_output.csv', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, os.path.basename(transformer.output_path)+'/sample_input.csv.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


### 4. Clean-up

#### A. Delete the model

In [44]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.
